# Graphs

This module contains the implementations of graphs (directed graphs).

The graphs we use when executing commands are called `term_graph`, `parse_graph`, and `dependency_graph`. the parse
graph is an abstract syntax tree. it contains nodes which represent commands, like `add_fact`, `rule`, `query`,
etc. In order to compute rule nodes, or compute query nodes, we use the term graph.

The term graph is holding the connection between all the relations and rules in the program, for more description read
`TermGraph`'s documentation.

The dependency graph stores dependencies between relations in the program, it is used by the term graph to recognize
mutually recursive relations. for more information read `DependencyGraph`'s documentation.

In [ ]:
#| default_exp graphs

In [ ]:
#| hide
from nbdev.showdoc import show_doc
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from __future__ import annotations

In [ ]:
#| hide
from nbdev.showdoc import show_doc

In [ ]:
#| export
#| output: false
from collections import OrderedDict
from enum import Enum
from pathlib import Path

import networkx as nx
from abc import ABC, abstractmethod, ABCMeta
from itertools import count
from typing import Set, List, Dict, Iterable, Union, Optional, OrderedDict as OrderedDictType, no_type_check, Any, Sequence
from spannerlib.ast_node_types import Relation, Rule, IERelation
from spannerlib.general_utils import get_input_free_var_names, get_output_free_var_names, get_free_var_to_relations_dict
from spannerlib.utils import patch_method,uniq_id

### Visualizations

In [ ]:
#| export 
import base64
from IPython.display import Image, display


In [ ]:
#| export
from copy import deepcopy

#### Mermaid

In [ ]:

#| export
def mm_ink(graphbytes):
    """Given a bytes object holding a Mermaid-format graph, return a URL that will generate the image."""
    base64_bytes = base64.b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    return "https://mermaid.ink/img/" + base64_string


def mm_display(graphbytes):
    """Given a bytes object holding a Mermaid-format graph, display it."""
    display(Image(url=mm_ink(graphbytes)))


def mm(graph):
    """Given a string containing a Mermaid-format graph, display it."""
    graphbytes = graph.encode("ascii")
    mm_display(graphbytes)


def mm_link(graph):
    """Given a string containing a Mermaid-format graph, return URL for display."""
    graphbytes = graph.encode("ascii")
    return mm_ink(graphbytes)


def mm_path(path):
    """Given a path to a file containing a Mermaid-format graph, display it"""
    with open(path, "rb") as f:
        graphbytes = f.read()
    mm_display(graphbytes)


In [ ]:
mm("""
graph LR;
A[hello]
A--> B & C & D;
B--> A & E;
C--> A & E;
D--> A & E;
E--> B & C & D;
""")

#### Jinja

In [ ]:
#| export
from jinja2 import Template, Environment, PackageLoader, meta

def template_undeclared_vars(template):
    """Computes all undeclared vars in a jinja template

    Args:
        template (Path or str): Path to file of template or string with the template content

    Returns:
        set: set of all undeclared vars
    """
    if isinstance(template, Path):
        template = template.read_text()
    env = Environment()
    parsed_content = env.parse(template)
    return meta.find_undeclared_variables(parsed_content)

def render_jinja(template, params: dict, silent=True, to_file: Path = None):
    """renders a jinja template

    Args:
        template (Path or str): Path to file of template or string with the template content
        params (Dict): parameter dictionary with the variables to render into the template
        silent (Bool, Optional): Whether to print the rendered template to screen, defaults to False
        to_file (Path, Optional): If a path is supplied, prints the template to the file of said path

    Returns:
        set: set of all undeclared vars
    """
    if isinstance(template, Path):
        template = template.read_text()
    instance_str = Template(template).render(**params)

    if not silent:
        print(instance_str)

    if to_file:
        to_file.write_text(instance_str)
        return None
    else:
        return instance_str

#### Drawing nx graphs

In [ ]:
#| export
graph_template = """
flowchart TB
{% for name,desc in nodes -%}
{{name}}["{{desc}}"]
{% endfor -%}

{% for s,t,desc in edges -%}
{% if desc -%}
{{s}} -.->|"{{desc}}"| {{t}}
{% else -%}
{{s}} -.-> {{t}}
{% endif -%}
{% endfor -%}
"""

def draw(g:nx.DiGraph,props=None,ret_mermaid=False):
    global graph_template

    def get_node_description(node,data):
        if props is None:
            attrs = ', '.join([f'{k}={v}' for k,v in data.items()])
        else :
            attrs = ', '.join([f'{k}={v}' for k,v in data.items() if k in props])

        return f'{node}\n{attrs}'

    nodes = [(n, get_node_description(n,data)) for n,data in g.nodes(data=True)]
    edges = [(u,v,None) for u,v,data in g.edges(data=True)]

    mermaid_text = render_jinja(graph_template,{'nodes':nodes,'edges':edges})
    if ret_mermaid:
        return mermaid_text
    mm(mermaid_text)

In [ ]:
# Create a directed graph
G = nx.DiGraph()

# Add nodes with different attributes
G.add_node(1, color='blue', size=10)
G.add_node(2, color='red', size=20)
G.add_node(3, color='green', size=30)

# Add edges
G.add_edge(1, 2)
G.add_edge(2, 3)


In [ ]:
#print(draw(G,ret_mermaid=True))
draw(G)

In [ ]:
#print(draw(G,ret_mermaid=True))
draw(G,props=['color'])

## Term Graph

In [ ]:
from spannerlib.lark_passes import (RemoveTokens, FixStrings, CheckReservedRelationNames,
                                              ConvertSpanNodesToSpanInstances, ConvertStatementsToStructuredNodes,
                                              CheckDefinedReferencedVariables,
                                              CheckReferencedRelationsExistenceAndArity,
                                              CheckReferencedIERelationsExistenceAndArity, CheckRuleSafety,
                                              TypeCheckAssignments, TypeCheckRelations,
                                              SaveDeclaredRelationsSchemas, ResolveVariablesReferences,
                                              ExecuteAssignments, AddStatementsToNetxParseGraph, GenericPass)
from spannerlib.grammar import parse_spannerlog

In [ ]:
from spannerlib.ast_node_types import Relation, Rule, IERelation,RelationDeclaration
from spannerlib.primitive_types import Span, DataTypes, DataTypeMapping


In [ ]:
# TODO from here, change all old code to just a netx graph and see what breaks
# make the term graph, parse graph etc. all work with just a nx graph

In [ ]:
def _compute_bounding_graph(relations: Set[Relation], # set of the regular relations in the rule body
                                ie_relations: Set[IERelation] # set of the ie relations in the rule body
                                ) -> OrderedDictType[IERelation, Set[Union[Relation, IERelation]]]: # a dictionary that maps each ie function to a set of it's bounding relations
    """
    This function gets body relations of a rule and computes for each ie relation the relations that bound it.
    """

    # holds the ie relation that are bounded
    bounded_ie_relations: Set[IERelation] = set()

    # maps each ie relation to it's bounding relations
    bounding_graph = OrderedDict()

    def find_bounding_relations_of_ie_function(ie_rel: IERelation) -> (
            Optional[Set[Union[Relation, IERelation]]]):
        """
        Finds all the relation that are already bounded that bind the ie relation.

        @param ie_rel: the ie relation to bound.
        @return: set of the bounding relations.
        """

        bounded_vars: Set[str] = set()
        bounding_relations_ = set()  # the trailing underscore is used to avoid shadowing the outer scope
        ie_input_terms = get_input_free_var_names(ie_rel)

        # iterate over all the bounded relations
        for relation in (relations | bounded_ie_relations):
            rel_terms = get_output_free_var_names(relation)
            # check if the relation and the ie relation have some common free vars
            mutual_vars = rel_terms.intersection(ie_input_terms)
            if len(mutual_vars) > 0:
                bounding_relations_.add(relation)
                bounded_vars = bounded_vars.union(mutual_vars)

        # check whether all ie relation's free vars are bounded
        if bounded_vars == ie_input_terms:
            return bounding_relations_
        else:
            # the ie relation can't be bounded yet
            return None

    # The function will eventually stop since the rule is safe.
    while True:
        # find the unbounded ie relations
        unbounded_ie_relations = ie_relations.difference(bounded_ie_relations)
        if len(unbounded_ie_relations) == 0:
            # all the ie relation are bounded
            break

        for ie_relation in unbounded_ie_relations:
            bounding_relations = find_bounding_relations_of_ie_function(ie_relation)
            if bounding_relations is not None:
                # we managed to bind the ie relation
                bounding_graph[ie_relation] = bounding_relations
                bounded_ie_relations.add(ie_relation)

    return bounding_graph

In [ ]:
def add_rule_to_term_graph(rule,t:nx.digraph):
    
    if isinstance(rule,RelationDeclaration):
        t.add_node(rule.relation_name,op='GetRelation')
        return
    if not isinstance(rule,Rule):
        raise ValueError('rule must be a Rule or RelationDeclaration instance')

    # from here its a rule    
    

    rule_head = rule.head_relation.relation_name
    t.add_node(rule_head,op='Union')

    relations,ie_relations = rule.get_relations_by_type()
    bounding_graph = _compute_bounding_graph(relations, ie_relations)

    # add projection node to vars of head relation
    project_id = f'project_head_{uniq_id()}'
    t.add_node(project_id,op='Project',vars=get_output_free_var_names(rule.head_relation))
    t.add_edge(rule_head,project_id)
    
    # for each relation in the body, put its node in the graph and add a projection node if not all vars are free
    for relation in relations:
        # no need to add a get relation node since it should be already defined in the term graph

        # if there are constants in the relation, add a select node        
        if get_output_free_var_names(relation) != get_input_free_var_names(relation):
            select_id = f'select_{relation}_{uniq_id()}'
            t.add_node(select_id,op='select',vars=get_const_vars(relation))
            t.add_edge(select_id,relation)

    # for each ie function in the body
        # make a calc node for it
        # put as input the joins of all the bounding relations
    # put a join node between all the relations and ie relations and the project node



#### Tests

In [ ]:
pass_list = [
    ConvertStatementsToStructuredNodes,
]

def run_passes(ast):
    global pass_list
    lark_tree = ast
    for curr_pass in pass_list:
        curr_pass_object = curr_pass()
        new_tree = curr_pass_object.run_pass(tree=lark_tree)
        if new_tree is not None:
            lark_tree = new_tree
            #logger.debug(f"lark tree after {curr_pass.__name__}:\n{lark_tree.pretty()}")
    return lark_tree


def _parse_and_clean(code):
    parse_tree = parse_spannerlog(code)
    cleaned_tree = [
        run_passes(statement) for statement in parse_tree.children
    ]
    return [x.children[0] for x in cleaned_tree if x is not None]


In [ ]:
input_rules = _parse_and_clean("""
        new A(str, str)
        new B(str, str)
        C(X, Y) <- A(X, Y)
        D(X, Y, X) <- C(X, Y)
        D(X, Y, Z) <- A(X, "1"), B(X, Z), ID(X) -> (Y), ID2(Y)->(Z,W)
""")
input_rules

[RelationDeclaration(A, [<DataTypes.string: 0>, <DataTypes.string: 0>]),
 RelationDeclaration(B, [<DataTypes.string: 0>, <DataTypes.string: 0>]),
 Rule(C(X, Y) <- A(X, Y)),
 Rule(D(X, Y, X) <- C(X, Y)),
 Rule(D(X, Y, Z) <- A(X, ""1""), B(X, Z), ID(X) -> (Y), ID2(Y) -> (Z, W))]

In [ ]:
rule = input_rules[-1]

In [ ]:
rule.body_relation_list

[Relation(A(X, ""1"")),
 Relation(B(X, Z)),
 IERelation(ID(X) -> (Y)),
 IERelation(ID2(Y) -> (Z, W))]

In [ ]:
s = get_output_free_var_names(rule.head_relation)
s

{'X', 'Y', 'Z'}

In [ ]:
t = list(s)[0]
t

Token('UPPER_CASE_NAME', 'Y')

In [ ]:
t.value

'Y'

In [ ]:
term_graph = nx.DiGraph()

term_graph.add_node('A',op='GetRelation')
term_graph.add_node('A',op='GetRelation')

In [ ]:
list(term_graph.nodes(data=True))

[('A', {'op': 'GetRelation'})]

In [ ]:
rule = input_rules[-1]
rule

Rule(D(X, Y, Z) <- A(X, ""1""), B(X, Z), ID(X) -> (Y), ID2(Y) -> (Z, W))

In [ ]:
relations = {r for r in rule.body_relation_list if isinstance(r,Relation)}
ie_relations = {r for r in rule.body_relation_list if isinstance(r,IERelation)}
relations,ie_relations

({Relation(A(X, ""1"")), Relation(B(X, Z))},
 {IERelation(ID(X) -> (Y)), IERelation(ID2(Y) -> (Z, W))})

In [ ]:
dict(_compute_bounding_graph(relations,ie_relations))

{IERelation(ID(X) -> (Y)): {Relation(A(X, ""1"")), Relation(B(X, Z))},
 IERelation(ID2(Y) -> (Z, W)): {IERelation(ID(X) -> (Y))}}

In [ ]:
# TODO build a new term graph with an nx graph and plot it

In [ ]:
for r in relations|ie_relations:
    print(get_input_free_var_names(r),get_output_free_var_names(r))

{Token('UPPER_CASE_NAME', 'X')} {Token('UPPER_CASE_NAME', 'Y'), Token('UPPER_CASE_NAME', 'X')}
set() {Token('UPPER_CASE_NAME', 'X')}
set() {Token('UPPER_CASE_NAME', 'Z'), Token('UPPER_CASE_NAME', 'X')}
{Token('UPPER_CASE_NAME', 'Y')} {Token('UPPER_CASE_NAME', 'Y'), Token('UPPER_CASE_NAME', 'Z'), Token('UPPER_CASE_NAME', 'W')}


### Old stuff

In [ ]:
#| export
PRETTY_INDENT = " " * 4
ROOT_NODE_ID = "__spannerlog_root"
ROOT_TYPE = "root"
TYPE = "type"
STATE = "state"
VALUE = "value"

In [ ]:
#| export
class EvalState(Enum):
    """
    will be used to determine if a term is computed or not.
    """

    NOT_COMPUTED = "not_computed"
    VISITED = "visited"
    COMPUTED = "computed"

    def __str__(self) -> str:
        return self.value        

In [ ]:
#| export
class TermNodeType(Enum):
    """
    will be used to represent type of term graph nodes.
    """

    SELECT = "select"
    JOIN = "join"
    PROJECT = "project"
    UNION = "union"
    CALC = "calc"
    RULE_REL = "rule_rel"
    GET_REL = "get_rel"

    def __str__(self) -> str:
        return self.value

In [ ]:
#| export
NodeIdType = Union[int, str]

In [ ]:
#| export
class GraphBaseOld(ABC):
    """
    This is an interface for a simple graph.
    """
    NodeIdType = Union[int, str]
    def __init__(self) -> None:
        self._visited_nodes: Set = set()

    @abstractmethod
    def add_node(self, 
                 node_id: Optional[NodeIdType] = None, # the id of the node (optional)
                  **attr: Dict # the attributes of the term
                  ) -> NodeIdType: # a node id that refers to the node that was added
        """
        Adds a node to the graph.
        """
        pass

    @abstractmethod
    def get_root_id(self) -> NodeIdType:
        """
        @return: the node id of the root of the graph.
        """
        pass

    @abstractmethod
    def remove_node(self,
                     node_id: NodeIdType # the id of the node that will be removed
                     ) -> None:
        """
        Removes a node from the graph.
        """
        pass

    @abstractmethod
    def add_edge(self, 
                 father_id: NodeIdType, # the id of the father node
                 son_id: NodeIdType, # the id of the son node
                 **attr: Dict # the attributes for the edge
                 ) -> None:
        """
        Adds the edge (`father_id`, `son_id`) to the graph. <br>
        the edge signifies that the father node is dependent on the son node.
        """
        pass

    @abstractmethod
    def pre_order_dfs_from(self, 
                           node_id: NodeIdType # the id of the node we will start the pre_order dfs from
                           ) -> Iterable[NodeIdType]: # an iterable of the node ids generated from a depth-first-search pre-ordering starting at the root of the graph
        pass

    def pre_order_dfs(self) -> Iterable[NodeIdType]:
        """
        @return: an iterable of the node ids generated from a depth-first-search pre-ordering starting at the root
        of the graph.
        """
        return self.pre_order_dfs_from(self.get_root_id())

    @abstractmethod
    def post_order_dfs_from(self,
                             node_id: NodeIdType # the id of the node we will run post_order dfs to
                             ) -> Iterable[NodeIdType]: # an iterable of the node ids generated from a depth-first-search post-ordering starting at the root of the graph.
        pass

    def post_order_dfs(self) -> Iterable[NodeIdType]:
        """
        @return: an iterable of the node ids generated from a depth-first-search post-ordering starting at the root
        of the graph.
        """
        return self.post_order_dfs_from(self.get_root_id())

    @abstractmethod
    def get_children(self,
                      node_id: NodeIdType # the id of the node that we wish to get its children
                      ) -> Iterable[NodeIdType]: # an iterable of the children of the node
        """
        In a term graph the children of a node are its dependencies
        this function returns the children of a node.
        """
        pass

    def get_child(self,
                   node_id: NodeIdType # the id of the node that we wish to get its child
                   ) -> NodeIdType: # the child of the node
        """
        @raise: RuntimeError if `node_id` has more than one child
        """
        children = list(self.get_children(node_id))
        if len(children) == 1:
            return children[0]

        raise RuntimeError(f"Expected one child, but node has {len(children)} children.")

    @abstractmethod
    def get_parents(self,
                     node_id: NodeIdType # the id of the node that we wish to get its predecessors
                     ) -> Iterable[NodeIdType]: # the parents (predecessors) of the node
        pass

    def get_parent(self,
                    node_id: NodeIdType # the id of the node that we wish to get its predecessor
                    ) -> NodeIdType: # the parent of the node
        """
        @raise: RuntimeError if `node_id` has more than one parent
        """
        parents = list(self.get_parents(node_id))
        if len(parents) == 1:
            return parents[0]

        raise RuntimeError(f"Expected one parent, but node has {len(parents)} parents.")

    @abstractmethod
    def is_contains_node(self,
                          node_id: NodeIdType # the node to look for
                          ) -> bool: # True if node is in the graph, False otherwise
        """
        Checks if node is in the graph.
        """
        pass

    @abstractmethod
    def set_node_attribute(self,
                            node_id: NodeIdType, # the id of the node
                              attr_name: str, # the name of the attribute
                                attr_value: Any # the value that will be set for the attribute
                                ) -> None:
        """
        Sets an attribute of a node.
        """
        pass

    @abstractmethod
    def get_node_attributes(self,
                             node_id: NodeIdType # the id of the node we wish to get its attributes
                             ) -> Dict[str, Any]: # a dict containing the attributes of the node
        pass

    @abstractmethod
    def _get_node_string(self, node_id: NodeIdType) -> str:
        """
        A utility function for __str__.

        @param node_id: a node id.
        @return: a string representation of the node.
        """
        pass

    def get_all_nodes_with_attributes(self,
                                       sub_graph_root: Optional[NodeIdType] = None, # the sub-graph root. if set to None then we search in the entire graph
                                         **attributes: Any # all the attributes name and values to look for
                                         ) -> Iterable[NodeIdType]: # all nodes that contains all the attributes inside the sub-graph

        root = self.get_root_id() if sub_graph_root is None else sub_graph_root
        node_ids = self.post_order_dfs_from(root)

        def is_node_contains_all_attributes(node_id: GraphBaseOld.NodeIdType) -> bool:
            node_attrs = self.get_node_attributes(node_id)
            return all(item in node_attrs.items() for item in attributes.items())

        return list(filter(is_node_contains_all_attributes, node_ids))

    def _pretty_aux(self, node_id: NodeIdType, level: int) -> List[str]:
        """
        A helper function for pretty().

        @param node_id: an id of a term in the term graph.
        @param level: the depth of the term in the tree (used for indentation).
        @return: a list of strings that represents the term and its children.
        """

        # get a representation of the node
        ret = [PRETTY_INDENT * level, self._get_node_string(node_id), '\n']

        if node_id in self._visited_nodes:
            return ret

        self._visited_nodes.add(node_id)

        # get a representation of the node's children
        for child_id in self.get_children(node_id):
            ret += self._pretty_aux(child_id, level + 1)

        return ret

    def pretty(self) -> str:
        """
        Prints a representation of the networkx tree.
        Works similarly to lark's pretty() function.

        example:
        for a computed query term of id 4 "?A(X)", this method will print
        (4) (computed) query: A(X)
        """

        self._visited_nodes = set()
        return ''.join(self._pretty_aux(self.get_root_id(), 0))
    


    def __str__(self) -> str:
        return self.pretty()

    def __getitem__(self, node_id: NodeIdType) -> Dict:
        return self.get_node_attributes(node_id)


In [ ]:
#| export
class NetxGraphOld(GraphBaseOld):
    """
    Implementation of a graph using a NetworkX graph. <br>
    The official documentation for NetworkX can be found here: [NetworkX Documentation](https://networkx.org/documentation/stable/index.html). <br>
    For a basic tutorial on NetworkX, you can visit: [NetworkX Introduction](https://networkx.org/documentation/stable//reference/introduction.html).
    """

    def __init__(self) -> None:
        super().__init__()
        # define the graph with 'DiGraph' to make sure the order of a reported node's children is the same
        # as the order they were added to the graph.
        self._graph = nx.DiGraph()

        # when a new node is added to the graph, it needs to have an id that was not used before
        # this field will serve as a counter that will provide a new term id
        self._node_id_counter: count[int] = count()

        # create the root of the graph. it will be used as a source for dfs/bfs
        self._root_id = self.add_node(node_id=ROOT_NODE_ID, type=ROOT_TYPE)

        # used for keep track of the printed nodes (in pretty function)
        self._visited_nodes = set()
    

    def add_node(self, node_id: Optional[GraphBaseOld.NodeIdType] = None, **attr: Any) -> GraphBaseOld.NodeIdType:
        # get the id for the new node (if id wasn't passed)
        node_id = next(self._node_id_counter) if node_id is None else node_id

        # add the new node to the graph and return its id
        self._graph.add_node(node_for_adding=node_id, **attr)
        return node_id

    def get_root_id(self) -> GraphBaseOld.NodeIdType:
        return self._root_id

    def remove_node(self, node_id: GraphBaseOld.NodeIdType) -> None:
        self._graph.remove_node(node_id)

    def add_edge(self, father_id: GraphBaseOld.NodeIdType, son_id: GraphBaseOld.NodeIdType, **attr: Any) -> None:

        # assert that both nodes are in the term graph
        if father_id not in self._graph.nodes:
            raise ValueError(f'father node of id {father_id} is not in the graph')
        if son_id not in self._graph.nodes:
            raise ValueError(f'son node of id {son_id} is not in the graph')

        # add an edge that represents the dependency of the father node on the son node
        self._graph.add_edge(father_id, son_id, **attr)

    def pre_order_dfs_from(self, node_id: GraphBaseOld.NodeIdType) -> Iterable[GraphBaseOld.NodeIdType]:
        return nx.dfs_preorder_nodes(self._graph, node_id)

    def post_order_dfs_from(self, node_id: GraphBaseOld.NodeIdType) -> Iterable[GraphBaseOld.NodeIdType]:
        return nx.dfs_postorder_nodes(self._graph, node_id)

    def get_children(self, node_id: GraphBaseOld.NodeIdType) -> Sequence[GraphBaseOld.NodeIdType]:
        return list(self._graph.successors(node_id))

    def set_node_attribute(self, node_id: GraphBaseOld.NodeIdType, attr_name: str, attr_value: Any) -> None:
        self._graph.nodes[node_id][attr_name] = attr_value

    def get_node_attributes(self, node_id: GraphBaseOld.NodeIdType) -> Dict[str, Any]:
        #return deepcopy(self._graph.nodes[node_id])
        return self._graph.nodes[node_id].copy()

    def _get_node_string(self, node_id: GraphBaseOld.NodeIdType) -> str:
        node_attrs = self.get_node_attributes(node_id)

        # get a string of the node's value (if it exists)
        if VALUE in node_attrs:
            term_value_string = f": {node_attrs[VALUE]}"
        else:
            term_value_string = ''

        # create a string representation of the node and return it
        term_string = f"({node_id}) {term_value_string}"
        return term_string

    def is_contains_node(self, node_id: GraphBaseOld.NodeIdType) -> bool:
        return self._graph.has_node(node_id)

    def get_parents(self, node_id: GraphBaseOld.NodeIdType) -> Iterable[GraphBaseOld.NodeIdType]:
        return self._graph.predecessors(node_id)

In [ ]:
#| export
class NetxStateGraphOld(NetxGraphOld):
    """
    This is a wrapper to NetxGraph that stores a state and type for each node in the graph.
    (This class will be the base class of the term graph and the parse graph while NetxGraph
    will be the base of dependency graph).
    """

    def __init__(self) -> None:
        super().__init__()

    def add_node(self, node_id: Optional[GraphBaseOld.NodeIdType] = None, **attr: Any) -> GraphBaseOld.NodeIdType:
        # assert the node has a type
        if TYPE not in attr:
            raise Exception("cannot add a node without a type")

        # if the node does not have a 'state' attribute, give it a default 'not computed' state
        if STATE not in attr:
            attr[STATE] = EvalState.NOT_COMPUTED

        return super(NetxStateGraphOld, self).add_node(node_id, **attr)

    def add_edge(self, father_id: GraphBaseOld.NodeIdType, son_id: GraphBaseOld.NodeIdType, **attr: Any) -> None:
        super(NetxStateGraphOld, self).add_edge(father_id, son_id, **attr)

        # if the son node is not computed, mark all of its ancestor as not computed as well
        son_term_state = self._graph.nodes[son_id][STATE]
        if son_term_state is EvalState.NOT_COMPUTED:
            # get all of the ancestors by reversing the graph and using a dfs algorithm from the son term
            ancestors_graph = nx.dfs_tree(self._graph.reverse(), source=son_id)
            ancestors_ids = list(ancestors_graph.nodes)
            # mark all of the ancestors as not computed
            for ancestor_id in ancestors_ids:
                self._graph.nodes[ancestor_id][STATE] = EvalState.NOT_COMPUTED

    def _get_node_string(self, node_id: GraphBaseOld.NodeIdType) -> str:
        node_attrs = self.get_node_attributes(node_id)

        # get a string of the node's value (if it exists)
        if VALUE in node_attrs:
            term_value_string = f": {node_attrs[VALUE]}"
        else:
            term_value_string = ''

        # create a string representation of the node and return it
        term_string = f"({node_id}) ({node_attrs[STATE]}) {node_attrs[TYPE]}{term_value_string}"
        return term_string

::: {.callout-note collapse="true"}

##### Example

In [ ]:
graph = NetxStateGraphOld()

node0 = graph.add_node(type="int")
node1 = graph.add_node(type="string")
node2 = graph.add_node(type="float")

graph.add_edge(node0,node1)
graph.add_edge(node0,node2)

print(graph._get_node_string(node0))

(0) (not_computed) int


In [ ]:
print(graph.get_children(node0))

[1, 2]


:::

In [ ]:
#| export
class DependencyGraphOld(NetxGraphOld):
    """
    The `DependencyGraph` class is designed to map and manage dependencies between rule relations in an spannerlog program. Each rule relation in the program corresponds to a node in this graph, with the node's ID being the name of the rule relation.

    ### Example

    Consider the following spannerlog program:

    ```prolog
    new A(int)
    B(X) <- A(X)
    C(X) <- A(X)
    B(X) <- C(X)  # B is dependent on C
    C(X) <- B(X)  # C is dependent on B
    D(X) <- C(X)  # D is dependent on C
    ```

    In this case the dependency graph will be: <br>
    **Nodes** = {A, B, C, D} (all the rule relations) <br>
    **Edges** = {(B, C), (C, B), (D, C)}
    """
    
    

    def __init__(self) -> None:
        super().__init__()

    def _add_relation(self, relation: Relation) -> None:
        """
        Adds relation to dependency graph.

        @param relation: the relation to add (should be rule relation).
        @return: the id of the relation node.
        """

        if self.is_contains_node(relation.relation_name):
            return

        self.add_node(node_id=relation.relation_name)
        self.add_edge(self._root_id, relation.relation_name)

    def is_dependent(self, head_rel: Relation, body_rel: Relation) -> bool:
        """
        Finds out whether the head relation is dependent in body relation, i.e. there is an edge from head_rel node to
        body_rel node).

        Example:
            A(X, Y) <- B(X, Y), C(X, Y)
            B(X, Y) <- A(X, Y)

            is_dependent(A, B), is_dependent(A, C) and is_dependent(B, A) will return True.
            On the other hand, is_dependent(C, A) will return False.

        @param head_rel: the head relation.
        @param body_rel: a body relation.
        @return: True if they are dependent, False otherwise.
        """

        if head_rel.relation_name == body_rel.relation_name:
            return False

        common_free_vars = set(head_rel.term_list).intersection(body_rel.term_list)
        return self.is_contains_node(body_rel.relation_name) and len(common_free_vars) > 0

    def add_dependencies(self, head_relation: Relation, body_relations: Set[Relation]) -> None:
        """
        Adds all the dependencies of the rule to the graph.

        @param head_relation: the head relation of the rule.
        @param body_relations: a set of rule's body relations.
        """

        self._add_relation(head_relation)

        for body_relation in body_relations:

            # add edge only if there is at least one free var in relation
            if self.is_dependent(head_relation, body_relation):
                edge = (head_relation.relation_name, body_relation.relation_name)
                num_of_edges = 1 + self._graph.get_edge_data(*edge, default={"amount": 0})["amount"]
                self.add_edge(*edge, amount=num_of_edges)

    def remove_relation(self, relation_name: str) -> None:
        """
        Removes the relation node from the dependency graph.

        @param relation_name: the name of the relation to remove.
        """

        self._graph.remove_node(relation_name)

    def remove_rule(self, rule: Rule) -> None:
        """
        Removes the dependencies of the rule from the graph.

        @param rule: the rule to remove.
        """

        head_relation = rule.head_relation

        body_relations, _ = rule.get_relations_by_type()
        for relation in body_relations:
            if self.is_dependent(head_relation, relation):
                edge = (head_relation.relation_name, relation.relation_name)
                num_of_edges = self._graph.get_edge_data(*edge)["amount"]
                if num_of_edges == 1:
                    self._graph.remove_edge(*edge)
                else:
                    self.add_edge(*edge, amount=num_of_edges - 1)

    def get_mutually_recursive_relations(self, relation_name: str) -> Set[str]:
        """
        Finds all relations that are mutually recursive with the input relation.

        @param relation_name: the name of the relation.
        @return: a set of relations names (including the input relation).
        """

        scc = nx.strongly_connected_components(self._graph)
        names_component, = filter(lambda component: relation_name in component, scc)
        return set(names_component)

    def _get_node_string(self, node_id: GraphBaseOld.NodeIdType) -> str:
        # for nicer printing format
        return str(node_id)

    def is_relation_in_use(self, relation_name: str) -> bool:
        """
        Checks if the relation is used by other existing rules.

        @param relation_name: name of the relation.
        @return: true if the node has parents (if node is root then we also return true).
        """
        # all the nodes are connected to global root
        predecessors_number = len(list(self._graph.predecessors(relation_name)))
        return predecessors_number > 1

    def __str__(self) -> str:
        return self.__class__.__name__ + " is:\n" + super().__str__()

::: {.callout-tip}

#### Main utility
In order to find mutually recursive rule relations we just need to compute the strongly connected components
of the graph (all the relations in a certain component are mutually recursive).

:::

In [ ]:
#| export
class TermGraphBaseOld(NetxStateGraphOld, metaclass=ABCMeta):
    """
    A wrapper to `NetxStateGraph` that adds utility functions which are independent
    of the structure of the term graph.
    """

    def __init__(self) -> None:
        super().__init__()
        # for each rule stores it's relevant nodes
        self._rule_to_nodes: Dict = dict()
        self._dependency_graph = DependencyGraphOld()

    @abstractmethod
    def add_rule_to_term_graph(self,
                 rule: Rule # the rule to add.
                 ) -> None:
        """
        Adds the rule to the term graph.
        This function is responsible for the structure of the term graph.
        """
        pass

    @abstractmethod
    def remove_rule(self,
                 rule: str # the rule to remove
                 ) -> bool: # Whether the deletion of the rule succeeded
        """
        Removes rule from the term graph.
        This function depends on the structure of the term graph.
        """
        pass

    def add_rule_node(self,
                  rule: Rule, # The rule to add
                  # all the nodes in the graph that are unique to the rule
                  nodes: Set[str] # (and thus should be removed if the rule is removed)
                  ) -> None:
        """
        Adds rule to term graph dict.
        """
        self._rule_to_nodes[str(rule)] = (rule, nodes)

    def _get_all_rules_with_head(self,
                                  relation_name: str # name of the relation
                                  ) -> List[str]: # a list of rules with rule head
        """
        Find all the rule with rule head.
        @raise ValueError: if relation name doesn't exist in the graph.
        """

        rules = [rule for rule in self._rule_to_nodes if rule.startswith(relation_name)]

        if len(rules) == 0:
            # if we are here then we didn't find the given relation
            raise ValueError(f"There are no relation with head '{relation_name}' in the term graph.")

        return rules

    def remove_rules_with_head(self, 
                               rule_head_name: str # a relation name
                               ) -> None:
        """
        Removes all rules with given rule head from the term graph. <br>
        @raise ValueError: if the `rule_head_name` is being used by another rule
        """

        rules = self._get_all_rules_with_head(rule_head_name)
        if self._dependency_graph.is_relation_in_use(rule_head_name):
            raise ValueError(f"The rule head '{rule_head_name}' can't be deleted since it's used in another existing "
                             f"rule.")

        for rule in rules:
            self.remove_rule(rule)

    def print_all_rules(self, 
                        head: Optional[str] # If you want to print all the rules with a specific head
                        ) -> None:
        """
        Prints all the registered rules.
        """

        if head is None:
            print("Printing all the rules:")
        else:
            print(f"Printing all the rules with head {head}:")

        i = 0
        for rule, _ in self._rule_to_nodes.values():
            if head is None or rule.head_relation.relation_name == head:
                print(f"\t{i + 1}. {rule}")
                i += 1

    def get_mutually_recursive_relations(self, 
                                         relation_name: str # relation name that you want to get its mutually recursive relations
                                         ) -> Set[str]: # A set of mutually recursive relations with the given relation name
        """
        See documentation of `get_mutually_recursive_relations` in `DependencyGraph`
        """
        return self._dependency_graph.get_mutually_recursive_relations(relation_name)

    def __str__(self) -> str:
        return super().__str__() + "\n" + str(self._dependency_graph)


# Old

In [ ]:
#| export
class TermGraphOld(TermGraphBaseOld):
    """
        This class is designed to transform each rule node in an spannerlog program into an execution graph. These execution graphs are then added to a term graph. <br>

        The term graph serves to store relationships between the following entities: <br>
        1. **The Rule Head**: The leading variable or fact in a rule. <br>
        2. **The Body Rule Relations**: The components that make up the conditions of the rule. <br>
        3. **The Body Base Relations and IE Relations**: Relations that are intrinsic or part of external functions. <br>
        4. **Computation Paths of The Rule Head**: All possible paths that lead to the evaluation of the rule head.

        Here is a sample spannerlog program for illustration:

        ```prolog
        new A(int, int)
        new B(int, int)
        C(X, Y) <- A(X, Y)
        D(X, Y) <- C(X, Y)
        D(X, Y) <- A(X, 1), B(X, Y), ID(X) -> (Y)  # ID is some ie function
        ```

        We will explain the meaning of the four core entities in the term graph with respect to the rules of `D`

        1. **Rule Head**: The rule head for these rules is `D(X, Y)`.
        2. **Body Rule Relations**: The body relations for the first rule is `C(X, Y)`. The second rule has no body rule relations.
        3. **Base Relations**: The base relations for the second rule are `A(X, 1)` and `B(X, Y)`. The first rule has no base relations.
        4. **Computation Paths**: The computation paths consist of paths from both the first and second rule.

        **Term Graph Structure**

        Nodes:

        * **`rule_rel Node`**: Each rule relation has a corresponding node in the term graph. Every `rule_rel node` is connected to a global root.
        * **`union_node`**: Each `rule_rel node` connects to a `union_node`.
        * **`project_node`**: Starts each computation path, projecting the columns of the relation it receives. It is connected to the `union_node`.
        * **`join_node`**: This node is under the `project_node` and joins all the body relations of the rule. Special cases where this node isn't used include:
            - There's only one relation in the rule's body.
            - None of the body relations have free variables.
        * **`calc_node`**: Connects each IE relation in the rule's body to the `join_node`. This node itself connects to another `join_node` that links all the bounding relations of the IE relation.
        * **`get_rel node`**: Connects each rule relation in the rule's body to the `join_node`. This node is linked to the corresponding rule root.
        * **`select_node`**: Used for relations with the same free variable (e.g., `A(X, X)`) or relations with some constant value (e.g., `A(1, X)`). This node deals with filtering tuples and is connected to the `join_node`.

        * **Base Relations**: Each base relation is directly connected to the `join_node`.

        Here's a representation of the term graph for the spannerlog program discussed:

            global root

               rule_rel node (of C)
                   union node
                       project node
                           get_rel node (get A)  @note: there isn't join node since there is only one body relation.

               rule_rel node (of D)
                   union node
                       project node
                           get_rel node (get C)  @note: there isn't join node since there is only one body relation.
                               rule_rel node (of C)

                       project node
                           join node (join A, B and ID)
                               get_rel node (get B)
                               select_node (select from A)
                                   get_rel node (get A)
                               calc node (of ID)
                                   join node (join A and B)
                                       get_rel node (get B)         @note: this get_rel node is the same one from above.
                                       select_node (select from A)  @note: this select node is the same one from above.
                                           get_rel node (get A)
       """

    def __init__(self) -> None:
        super().__init__()
    
    @staticmethod
    def _compute_bounding_graph(relations: Set[Relation], # set of the regular relations in the rule body
                                 ie_relations: Set[IERelation] # set of the ie relations in the rule body
                                 ) -> OrderedDictType[IERelation, Set[Union[Relation, IERelation]]]: # a dictionary that maps each ie function to a set of it's bounding relations
        """
        This function gets body relations of a rule and computes for each ie relation the relations that bound it.
        """

        # holds the ie relation that are bounded
        bounded_ie_relations: Set[IERelation] = set()

        # maps each ie relation to it's bounding relations
        bounding_graph = OrderedDict()

        def find_bounding_relations_of_ie_function(ie_rel: IERelation) -> (
                Optional[Set[Union[Relation, IERelation]]]):
            """
            Finds all the relation that are already bounded that bind the ie relation.

            @param ie_rel: the ie relation to bound.
            @return: set of the bounding relations.
            """

            bounded_vars: Set[str] = set()
            bounding_relations_ = set()  # the trailing underscore is used to avoid shadowing the outer scope
            ie_input_terms = get_input_free_var_names(ie_rel)

            # iterate over all the bounded relations
            for relation in (relations | bounded_ie_relations):
                rel_terms = get_output_free_var_names(relation)
                # check if the relation and the ie relation have some common free vars
                mutual_vars = rel_terms.intersection(ie_input_terms)
                if len(mutual_vars) > 0:
                    bounding_relations_.add(relation)
                    bounded_vars = bounded_vars.union(mutual_vars)

            # check whether all ie relation's free vars are bounded
            if bounded_vars == ie_input_terms:
                return bounding_relations_
            else:
                # the ie relation can't be bounded yet
                return None

        # The function will eventually stop since the rule is safe.
        while True:
            # find the unbounded ie relations
            unbounded_ie_relations = ie_relations.difference(bounded_ie_relations)
            if len(unbounded_ie_relations) == 0:
                # all the ie relation are bounded
                break

            for ie_relation in unbounded_ie_relations:
                bounding_relations = find_bounding_relations_of_ie_function(ie_relation)
                if bounding_relations is not None:
                    # we managed to bind the ie relation
                    bounding_graph[ie_relation] = bounding_relations
                    bounded_ie_relations.add(ie_relation)

        return bounding_graph

In [ ]:
#| export
#| hide
@patch_method
def add_relation(self: TermGraphOld, 
                    relation: Relation # the relation to add
                    # returns the relation node id if is_rule is false
                    ) -> GraphBaseOld.NodeIdType: # otherwise returns the relation child node id (union node)
    """
    Adds the relation to the graph. if it's already inside nothing is done.

    @note: we assume the relation is rule relation and not declared relation.
    """

    relation_name = relation.relation_name
    if self.is_contains_node(relation_name):
        return self.get_relation_union_node(relation_name)

    self.add_node(node_id=relation_name, type=TermNodeType.RULE_REL, value=relation)
    self.add_edge(self.get_root_id(), relation_name)
    union_id: GraphBaseOld.NodeIdType = self.add_node(type=TermNodeType.UNION)
    self.add_edge(relation_name, union_id)

    return union_id

In [ ]:
#| export
#| hide
@patch_method
def get_relation_union_node(self: TermGraphOld, 
                            relation_name: str # name of a relation
                            ) -> GraphBaseOld.NodeIdType: # the union node of the given relation

    union_id, = self.get_children(relation_name)  # relation has only one child (the union node).
    return union_id

In [ ]:
#| export
#| hide
@patch_method
def add_rule_to_term_graph(self: TermGraphOld, 
                            rule: Rule # the rule to add
                            ) -> None:
    """
    Generates the execution tree of the rule and adds it to the term graph.
    Implements the following pseudo code:

    def generate_computation_graph(self, head, body):
        bounding_graph = find_bounding_graph(body)
        build_root
        connect_all_bodies_to_root_with_join
        for each ie_function:
        make calc_node
        connect to join of all bounding bodies
    """

    # maps each relation to it's node id in the term graph.
    relation_to_branch_id: Dict[Union[Relation, IERelation], int] = {}

    # stores the nodes that were added to to execution graph
    nodes = set()

    def add_node(node_id # a new node that was added
                    ) -> None:
        """
        Saves all the nodes that were added due to the rule.
        """
        nodes.add(node_id)

    def add_join_branch(head_id, # the node to which join node will be connected
                        joined_relations: Set[Union[Relation, IERelation]], # a set of relations
                        future_ie_relations: Optional[Set[IERelation]] = None # a set of ie relations that will be added to branch in the future
                        ) -> int: # the id of the join node
        """
        Connects all the relations to a join node. Connects the join_node to head_id.
        """

        future_ies = set() if future_ie_relations is None else future_ie_relations
        total_relations = joined_relations | future_ies

        # check if there is one relation (we don't need join)
        if len(total_relations) == 1 and len(joined_relations) == 1:
            add_relation_branch(next(iter(total_relations)), head_id)
            return head_id

        join_dict = get_free_var_to_relations_dict(total_relations)
        if not join_dict:
            return head_id

        # add join node
        join_node_id_ = self.add_node(type=TermNodeType.JOIN, value=join_dict)
        add_node(join_node_id_)

        self.add_edge(head_id, join_node_id_)
        for relation in joined_relations:
            add_relation_branch(relation, join_node_id_)

        return join_node_id_

    def add_relation_to(relation: Union[Relation, IERelation], # a relation
                        father_node_id: int # the node to which the relation will be connected
                        ) -> None:
        """
        Adds relation to father id.
        """

        get_rel_id = self.add_node(type=TermNodeType.GET_REL, value=relation)
        add_node(get_rel_id)

        # cache the branch
        relation_to_branch_id[relation] = get_rel_id
        self.add_edge(father_node_id, get_rel_id)

        # if relation is a rule relation we connect it to the root of the relation (rel_id)
        if self.is_contains_node(relation.relation_name):
            self.add_edge(get_rel_id, relation.relation_name)

    @no_type_check
    def add_relation_branch(relation: Union[Relation, IERelation], # a relation
                            join_node_id_: int # the join node to which the relation will be connected
                            ) -> None:
        """
        Adds relation to the join node.
        Finds all the columns of the relation that needed to be filtered and Adds select branch if needed.
        """

        # check if the branch already exists (if relations is ie relation the branch already exists)
        if relation in relation_to_branch_id:
            self.add_edge(join_node_id_, relation_to_branch_id[relation])
            return

        free_vars = get_output_free_var_names(relation)
        term_list = relation.get_term_list()

        # check if there is a constant (A("4")), or there is a free var that appears multiple times (A(X, X))
        if len(free_vars) != len(term_list) or len(term_list) != len(set(term_list)):
            # create select node and connect relation branch to it
            select_info = relation.get_select_cols_values_and_types()
            select_node_id = self.add_node(type=TermNodeType.SELECT, value=select_info)
            add_node(select_node_id)
            self.add_edge(join_node_id_, select_node_id)
            add_relation_to(relation, select_node_id)
            relation_to_branch_id[relation] = select_node_id
        else:
            # no need to add select node
            add_relation_to(relation, join_node_id_)

    def add_calc_branch(join_node_id_: int, # the join node to which the branch will be connected
                        ie_relation_: IERelation, # an ie relation
                        bounding_graph_: OrderedDict # the bounding graph of the ie relations
                        ) -> int: # the calc_node's id
        """
        Adds a calc branch of the ie relation.
        """
        calc_node_id_ = self.add_node(type=TermNodeType.CALC, value=ie_relation_)
        add_node(calc_node_id_)

        # join all the ie relation's bounding relations. The bounding relations already exists in the graph!
        # (since we iterate on the ie relations in the same order they were bounded).
        bounding_relations = bounding_graph_[ie_relation_]
        add_join_branch(calc_node_id_, bounding_relations)
        self.add_edge(join_node_id_, calc_node_id_)
        return calc_node_id_

    head_relation = rule.head_relation
    relations, ie_relations = rule.get_relations_by_type()
    # computes the bounding graph (it's actually an ordered dict).
    bounding_graph = TermGraphOld._compute_bounding_graph(relations, ie_relations)

    # make root
    union_id = self.add_relation(head_relation)
    project_id = self.add_node(type=TermNodeType.PROJECT, value=head_relation.term_list)
    self.add_edge(union_id, project_id)
    add_node(project_id)

    # connect all regular relations to join node
    join_node_id = add_join_branch(project_id, relations, ie_relations)

    # iterate over ie relations in the same order they were bounded
    for ie_relation in bounding_graph:
        calc_node_id = add_calc_branch(join_node_id, ie_relation, bounding_graph)
        relation_to_branch_id[ie_relation] = calc_node_id

    self.add_rule_node(rule, nodes)
    self._dependency_graph.add_dependencies(head_relation, relations)


In [ ]:
#| export
#| hide
@patch_method
def remove_rule(self: TermGraphOld, 
                rule: str # the rule to remove. unlike add_rule, here rule should be string as it is a user input
                ) -> bool: # true if the head relation was deleted, false otherwise
    """
    Removes a rule from term graph.

    @raise Exception if the rule doesn't exist in the term graph
    """

    if rule not in self._rule_to_nodes:
        raise ValueError(f"The rule '{rule}' was never registered "
                            f"(you can run 'print_all_rules' to see all the registered rules)")

    actual_rule, nodes = self._rule_to_nodes[rule]
    rule_name = actual_rule.head_relation.relation_name
    union_node = self.get_relation_union_node(rule_name)

    is_last_rule_path = len(list(self.get_children(union_node))) == 1
    is_rule_used = self._dependency_graph.is_relation_in_use(rule_name)

    # check if something is connected to the rel_root and the root is going to be deleted (this shouldn't happen)
    if is_last_rule_path and is_rule_used:
        raise RuntimeError(f"The rule '{rule}' can't be deleted since '{rule_name}' is used in another existing "
                            f"rule.")

    self._graph.remove_nodes_from(nodes)
    del self._rule_to_nodes[rule]

    self._dependency_graph.remove_rule(actual_rule)

    if is_last_rule_path:
        self._graph.remove_nodes_from((rule_name, union_node))
        self._dependency_graph.remove_relation(rule_name)
        return True

    return False

::: {.callout-note}
In some cases ie relation is bounded by other ie relation.
e.g. A(X) <- B(Y), C(Z) -> (X), D(Y) -> (Z); in this example C is bounded only by D.
:::